In [36]:
import json
import os

import hashlib
from typing import List, Dict, Any
from tqdm import tqdm
import sys
import concurrent.futures
import random

from get_text_embedding import get_text_embedding

from dotenv import load_dotenv
from openai import OpenAI
import os, requests

In [37]:
from rank_bm25 import BM25Okapi
import jieba, re

def _preprocess(doc: str) -> str:
    # 切分 6-8 位数字（日期/代码），弱化连字符/下划线
    doc = re.sub(r'(\d{6,8})', r' \1 ', doc)
    doc = doc.replace('-', ' ').replace('_', ' ')
    return doc

class FileBM25Index:
    """
    文件级 BM25：用 文件名 + 前3页文本 + 标题/目录 行 来增强“版本可分性”
    """
    def __init__(self, pages_per_file: int = 3, per_page_chars: int = 1500,
                 repeat_filename_tokens: int = 2):
        self.pages_per_file = pages_per_file
        self.per_page_chars = per_page_chars
        self.repeat_filename_tokens = repeat_filename_tokens
        self.files = []
        self.tokens = []
        self.fn_to_idx = {}
        self.bm25 = None

    def build(self, chunks):
        # 收集每文件的若干最早页 & 抓取标题行（MinerU markdown 常有 #/##/表/图）
        pages_by_file = {}
        titles_by_file = {}
        for c in chunks:
            fn = c["metadata"]["file_name"]
            pg = int(c["metadata"]["page"])
            txt = c["content"]
            # 标题/目录/图表行
            titles = "\n".join(
                ln for ln in txt.splitlines()
                if ln.lstrip().startswith(('#','##','表','图'))
            )
            if titles:
                titles_by_file.setdefault(fn, []).append(titles[:500])

            pages_by_file.setdefault(fn, {})
            # 只收每页一次
            if pg not in pages_by_file[fn]:
                pages_by_file[fn][pg] = txt

        self.files, self.tokens, self.fn_to_idx = [], [], {}
        for fn, pgmap in pages_by_file.items():
            first_pages = [pgmap[p] for p in sorted(pgmap)[: self.pages_per_file]]
            titles = "\n".join(titles_by_file.get(fn, []))[:1500]
            # 文档：文件名 + 标题/目录摘要 + 前3页各1500字
            doc = f"{fn}\n{titles}\n" + "\n".join(fp[: self.per_page_chars] for fp in first_pages)
            doc = _preprocess(doc)
            toks = list(jieba.cut(doc))
            # 额外重复文件名分词，放大“版本词/日期码”权重
            toks += list(jieba.cut(_preprocess(fn))) * self.repeat_filename_tokens

            self.fn_to_idx[fn] = len(self.files)
            self.files.append(fn)
            self.tokens.append(toks)

        self.bm25 = BM25Okapi(self.tokens)

    def top_files(self, query, n=25):
        if not self.bm25:
            return []
        q = list(jieba.cut(_preprocess(query)))
        scores = self.bm25.get_scores(q)
        order = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)
        return [(self.files[i], float(scores[i])) for i in order[:n]]


In [38]:
import re
def rrf(rank, k=90):
    """
    Reciprocal Rank Fusion 基本项。
    约定 rank 为 0-based，这里内部转为 1-based 以避免极端放大。
    k 越大越“温和”，推荐 90。
    """
    r = int(rank) + 1
    return 1.0 / (k + r)


THEME_TOKENS = ["二次创业","三条增长曲线","深度复盘","稳定成长","公司深度报告","再谈","走在修复的道路上"]
def extract_years(q): return set(re.findall(r"20\d{2}", q))

def soft_bias(ranked, question, year_boost=0.15, theme_boost=0.05):
    years = extract_years(question)
    themed = any(t in question for t in THEME_TOKENS)
    for c in ranked:
        fn = c["metadata"]["file_name"]
        s  = c.get("rerank_score", 0.0)
        if years and any(y in fn for y in years): s += year_boost
        if themed: s += theme_boost if any(t in fn for t in THEME_TOKENS) else 0.0
        c["rerank_score"] = s
    ranked.sort(key=lambda x: x["rerank_score"], reverse=True)
    return ranked



In [39]:
from collections import defaultdict
import numpy as np

def file_vote(ranked_chunks, top_n_files=1, agg="sum"):
    buckets = defaultdict(list)
    for c in ranked_chunks:
        fn = c["metadata"]["file_name"]
        s  = c.get("rerank_score", c.get("ret_score", 0.0))
        buckets[fn].append(float(s))

    def agg_fn(v):
        v = np.asarray(v, float)
        return v.sum() if agg=="sum" else (v.mean() if agg=="mean" else v.max())

    scored = sorted([(fn, agg_fn(v)) for fn, v in buckets.items()], key=lambda x: x[1], reverse=True)
    keep = set(fn for fn,_ in scored[:top_n_files])
    kept = [c for c in ranked_chunks if c["metadata"]["file_name"] in keep]
    return kept, (scored[0][0] if scored else None, dict(scored))


In [40]:
# # Build windows from existing MinerU chunks: all_pdf_page_chunks.json -> all_pdf_windows.json RUN ONCE FIX page
# import json, re, hashlib
# from pathlib import Path

# SRC = Path("./all_pdf_page_chunks_mineru.json")          # MinerU output you already have
# DST = Path("./all_pdf_windows_mineru.json")              # new, windowized chunks

# HDR = re.compile(r"^(#{1,3})\s+.+", re.M)         # #, ##, ### headings
# FIG = re.compile(r"!\[[^\]]*\]\([^)]+\)")         # markdown images
# TB  = re.compile(r"<table[\s\S]*?</table>", re.I)  # html tables

# def _norm(s): return re.sub(r"\s+", " ", (s or "").strip())

# def split_by_headers(md: str):
#     # keep figure/table blocks as atomic spans
#     blocks = []
#     # placeholder tokens for figures/tables
#     fig_tokens, tab_tokens = [], []
#     def _stash(pattern, text, token_prefix):
#         out, toks, i = text, [], 0
#         for m in pattern.finditer(text):
#             tok = f"__{token_prefix}_{i}__"
#             toks.append((tok, m.group(0)))
#             out = out.replace(m.group(0), tok, 1)
#             i += 1
#         return out, toks
#     text, figs = _stash(FIG, md, "FIG")
#     text, tabs = _stash(TB,  text, "TAB")

#     # split by headers; if none, single block
#     parts = []
#     last = 0
#     for m in HDR.finditer(text):
#         if m.start() > last:
#             parts.append(text[last:m.start()])
#         parts.append(text[m.start():m.end()])  # header line as its own piece
#         last = m.end()
#     if last < len(text):
#         parts.append(text[last:])

#     # restore tokens
#     def _restore(s, toks):
#         for tok, val in toks:
#             s = s.replace(tok, val)
#         return s
#     parts = [_restore(_restore(p, figs), tabs) for p in parts]
#     # drop empties
#     parts = [p for p in parts if _norm(p)]
#     return parts

# def windowize(parts, target=900, overlap=150):
#     out, buf = [], ""
#     for p in parts:
#         p = _norm(p)
#         if len(p) <= target:
#             if len(buf) + len(p) + 1 <= target:
#                 buf = (buf + " " + p).strip()
#             else:
#                 if buf: out.append(buf)
#                 tail = buf[-overlap:] if buf and overlap else ""
#                 buf = (tail + " " + p).strip()
#         else:
#             # long piece -> sliding windows
#             stride = max(50, target - overlap)
#             i = 0
#             while i < len(p):
#                 out.append(_norm(p[i:i+target]))
#                 i += stride
#     if buf: out.append(buf)
#     return out

# print("Loading MinerU page chunks…")
# chunks = json.loads(Path(SRC).read_text(encoding="utf-8"))
# print(f"Loaded {len(chunks)} page-chunks")

# seen, windows = set(), []
# for c in chunks:
#     file = c["metadata"]["file_name"]
#     page = int(c["metadata"]["page"])+1   # shift 1 , quick fix for the fucked up in mineru process
#     parts = split_by_headers(c["content"])
#     wins  = windowize(parts, target=900, overlap=150)
#     for widx, w in enumerate(wins):
#         h = hashlib.md5(_norm(w).encode()).hexdigest()
#         if h in seen: 
#             continue
#         seen.add(h)
#         windows.append({
#             "id": f"{c['id']}_w{widx}",
#             "content": w,
#             "metadata": {
#                 "file_name": file,
#                 "page": page,         
#                 "widx": widx
#             }
#         })

# print(f"Built {len(windows)} windows")
# DST.write_text(json.dumps(windows, ensure_ascii=False, indent=2), encoding="utf-8")
# print(f"Saved -> {DST}")

In [41]:
from collections import defaultdict
import numpy as np
#   Score drop IDK
# def file_vote(ranked_chunks, top_n_files=1, agg="sum"):
#     """
#     Aggregate scores per filename and keep chunks from the winning file(s).
#     ranked_chunks: reranked list (desc), each with ["metadata"]["file_name"] and "rerank_score".
#     """
#     buckets = defaultdict(list)
#     for c in ranked_chunks:
#         fn = c["metadata"]["file_name"]
#         s = c.get("rerank_score", c.get("ret_score", 0.0))
#         buckets[fn].append(float(s))

#     def agg_fn(v):
#         v = np.asarray(v, float)
#         if agg == "mean": return v.mean()
#         if agg == "max":  return v.max()
#         return v.sum()  # default

#     scored = sorted([(fn, agg_fn(v)) for fn, v in buckets.items()],
#                     key=lambda x: x[1], reverse=True)
#     keep_files = set(fn for fn, _ in scored[:top_n_files])
#     kept = [c for c in ranked_chunks if c["metadata"]["file_name"] in keep_files]
#     # Keep a tiny meta for telemetry
#     return kept, (scored[0][0] if scored else None, dict(scored))

def page_vote(ranked_chunks, top_m_pages=1, agg="max"):
    """
    ranked_chunks: list already reranked by your reranker (desc)
    Aggregate scores per (file_name, page) and return chunks from the winning page(s).
    """
    buckets = defaultdict(list)
    for c in ranked_chunks:
        key = (c["metadata"]["file_name"], int(c["metadata"]["page"]))
        s = c.get("rerank_score", c.get("ret_score", 0.0))
        buckets[key].append(float(s))

    def agg_fn(v):
        v = np.asarray(v, float)
        if agg == "sum": return v.sum()
        if agg == "mean": return v.mean()
        return v.max()

    scored_pages = sorted([(k, agg_fn(v)) for k, v in buckets.items()], key=lambda x: x[1], reverse=True)
    keep_keys = set(k for k,_ in scored_pages[:top_m_pages])
    return [c for c in ranked_chunks if (c["metadata"]["file_name"], int(c["metadata"]["page"])) in keep_keys]

def expand_neighbors_on_page(page_chunks, radius=1, max_total=8):
    # Assumes all chunks are from the same (file,page); use their widx to add neighbors
    page_chunks = sorted(page_chunks, key=lambda x: x["metadata"].get("widx", 0))
    out = []
    picked = set()
    for c in page_chunks[:max_total]:
        w = c["metadata"].get("widx", 0)
        for j in range(max(0, w - radius), min(len(page_chunks), w + radius + 1)):
            cj = page_chunks[j]
            key = cj["metadata"].get("widx", j)
            if key in picked: 
                continue
            picked.add(key); out.append(cj)
            if len(out) >= max_total: 
                return out
    return out or page_chunks[:max_total]


In [42]:
import os, re, time, requests

class SiliconFlowReranker:
    def __init__(self, api_key: str = None,
                 model: str = "BAAI/bge-reranker-v2-m3",
                 endpoint: str = "https://api.siliconflow.cn/v1/rerank",
                 timeout: int = 30,
                 max_retries: int = 4,
                 add_header: bool = True,
                 header_snippet_chars: int = 300):
        self.api_key = api_key or os.getenv('LOCAL_API_KEY')
        if not self.api_key:
            raise ValueError("Missing SILICONFLOW_API_KEY")
        self.model = model
        self.endpoint = endpoint
        self.timeout = timeout
        self.max_retries = max_retries

        # NEW
        self.add_header = add_header
        self.header_snippet_chars = header_snippet_chars

    def _extract_date_from_filename(self, fn: str) -> str:
        """
        Try to parse YYYY[-_.]?MM[-_.]?DD in filename and normalize to YYYY-MM-DD.
        Returns '' if not found.
        """
        m = re.search(r'(20\d{2})[-_.]?(0[1-9]|1[0-2])[-_.]?(0[1-9]|[12]\d|3[01])', fn)
        if not m: return ''
        y, mo, d = m.groups()
        return f"{y}-{mo}-{d}"

    def _make_header(self, c: dict) -> str:
        meta = c.get("metadata", {})
        fn = str(meta.get("file_name", ""))
        pg = str(meta.get("page", ""))
        dt = self._extract_date_from_filename(fn)
        snippet = c.get("content", "")[: self.header_snippet_chars]
        parts = [f"【文件】{fn}", f"【页】{pg}"]
        if dt: parts.append(f"【日期】{dt}")
        parts.append(f"【摘要】{snippet}")
        return "\n".join(parts) + "\n"

    def rerank(self, question: str, candidates: list, return_meta: bool = False):
        """
        Returns:
          ranked_chunks, meta where meta = {"status": http_status or None, "attempts": N}
        Retries on 429/5xx with exponential backoff.
        """
        # Build documents (with optional header)
        if self.add_header:
            documents = [self._make_header(c) + c["content"] for c in candidates]
        else:
            documents = [c["content"] for c in candidates]

        payload = {"model": self.model, "query": question, "documents": documents}
        headers = {"Authorization": f"Bearer {self.api_key}", "Content-Type": "application/json"}

        status = None
        attempts = 0
        data = None

        for attempt in range(self.max_retries):
            attempts = attempt + 1
            try:
                r = requests.post(self.endpoint, json=payload, headers=headers, timeout=self.timeout)
                status = r.status_code
                if status in (429, 500, 502, 503, 504):
                    time.sleep(2 ** attempt)  # backoff
                    continue
                r.raise_for_status()
                data = r.json()
                break
            except requests.RequestException:
                time.sleep(2 ** attempt)

        ranked = []
        if data:
            results = data.get("results") or data.get("data") or []
            seen = set()
            for res in results:
                idx = res.get("index")
                if idx is None or idx >= len(candidates):
                    continue
                seen.add(idx)
                c = dict(candidates[idx])
                c["rerank_score"] = float(res.get("relevance_score", 0.0))
                ranked.append(c)
            # fill any missing
            for i, c in enumerate(candidates):
                if i not in seen:
                    cc = dict(c); cc["rerank_score"] = 0.0
                    ranked.append(cc)
            ranked.sort(key=lambda x: x["rerank_score"], reverse=True)

        meta = {"status": status, "attempts": attempts}
        return (ranked, meta) if return_meta else ranked


In [43]:
class PageChunkLoader:
    def __init__(self, json_path: str):
        self.json_path = json_path
    def load_chunks(self) -> List[Dict[str, Any]]:
        with open(self.json_path, 'r', encoding='utf-8') as f:
            return json.load(f)

In [44]:
class EmbeddingModel:
    def __init__(self, batch_size: int = 64):
        self.api_key = os.getenv('LOCAL_API_KEY')
        self.base_url = os.getenv('LOCAL_BASE_URL')
        self.embedding_model = os.getenv('LOCAL_EMBEDDING_MODEL')
        self.batch_size = batch_size
        if not self.api_key or not self.base_url:
            raise ValueError('请在.env中配置LOCAL_API_KEY和LOCAL_BASE_URL')

    def embed_texts(self, texts: List[str]) -> List[List[float]]:
        return get_text_embedding(
            texts,
            api_key=self.api_key,
            base_url=self.base_url,
            embedding_model=self.embedding_model,
            batch_size=self.batch_size
        )

    def embed_text(self, text: str) -> List[float]:
        return self.embed_texts([text])[0]

In [45]:
class SimpleVectorStore:
    def __init__(self):
        self.embeddings = []
        self.chunks = []
    def add_chunks(self, chunks: List[Dict[str, Any]], embeddings: List[List[float]]):
        self.chunks.extend(chunks)
        self.embeddings.extend(embeddings)
    def search(self, query_embedding: List[float], top_k: int = 3) -> List[Dict[str, Any]]:
        from numpy import dot
        from numpy.linalg import norm
        import numpy as np
        if not self.embeddings:
            return []
        emb_matrix = np.array(self.embeddings)
        query_emb = np.array(query_embedding)
        sims = emb_matrix @ query_emb / (norm(emb_matrix, axis=1) * norm(query_emb) + 1e-8)
        idxs = sims.argsort()[::-1][:top_k]
        return [self.chunks[i] for i in idxs]

In [46]:
def soft_bias_filewise(ranked, question, year_boost=0.03, theme_boost=0.01):
    years = set(re.findall(r"20\d{2}", question))
    THEME_TOKENS = ["二次创业","三条增长曲线","深度复盘","稳定成长","公司深度报告","再谈","走在修复的道路上"]
    file_bonus = {}
    for c in ranked:
        fn = c["metadata"]["file_name"]
        if fn in file_bonus: 
            continue
        b = 0.0
        if years and any(y in fn for y in years): 
            b += year_boost
        if any(t in fn for t in THEME_TOKENS):   
            b += theme_boost
        file_bonus[fn] = b
    for c in ranked:
        fn = c["metadata"]["file_name"]
        c["rerank_score"] = float(c.get("rerank_score", 0.0)) + file_bonus.get(fn, 0.0)
    ranked.sort(key=lambda x: x["rerank_score"], reverse=True)
    return ranked

In [47]:
from collections import defaultdict

def pick_best_page_and_expand(file_scoped, final_k=5, radius=1):
    # group by page
    by_page = defaultdict(list)
    for c in file_scoped:
        by_page[int(c["metadata"]["page"])].append(c)
    # score each page by max rerank_score
    page_scores = [(pg, max(float(x.get("rerank_score", 0.0)) for x in cs))
                   for pg, cs in by_page.items()]
    best_pg, _ = max(page_scores, key=lambda t: t[1])

    page_chunks = sorted(by_page[best_pg], key=lambda x: x["metadata"].get("widx", 0))
    # expand around the highest-scored chunk on that page
    top_idx = max(range(len(page_chunks)),
                  key=lambda i: float(page_chunks[i].get("rerank_score", 0.0)))
    w0 = page_chunks[top_idx]["metadata"].get("widx", top_idx)

    # collect neighbors by widx
    wanted = {w0}
    for r in range(1, radius+1):
        wanted.add(w0 - r); wanted.add(w0 + r)

    chosen = [c for c in page_chunks if c["metadata"].get("widx", -10**9) in wanted]
    # backfill with other chunks on same page if needed
    if len(chosen) < final_k:
        for c in page_chunks:
            if c not in chosen:
                chosen.append(c)
            if len(chosen) >= final_k: break
    return chosen[:final_k]


In [48]:
class SimpleRAG:
    def __init__(
        self,
        chunk_json_path: str,
        model_path: str = None,
        batch_size: int = 32,
        use_rerank: bool = False,
        candidate_k: int = 120,
        final_k: int = 5,
        reranker=None,
        
    ):
        self.loader = PageChunkLoader(chunk_json_path)
        self.embedding_model = EmbeddingModel(batch_size=batch_size)
        self.vector_store = SimpleVectorStore()
        

        # Rerank controls
        self.use_rerank = use_rerank
        self.candidate_k = candidate_k
        self.final_k = final_k
        self.reranker = reranker
        

        # Behavior flags
        self.lock_source     = bool(int(os.getenv("LOCK_SOURCE",  "0")))
        self.strict_rerank   = bool(int(os.getenv("STRICT_RERANK","0")))
        self.debug_telemetry = bool(int(os.getenv("DEBUG_RAG",    "1")))

        self.limit_per_file = int(os.getenv("LIMIT_PER_FILE", "12"))


        # # Retrieval helpers
        # self.file_bm25 = FileBM25Index(pages_per_file=3, per_page_chars=1500, repeat_filename_tokens=2)
        # self.file_bm25.build(self.chunks)


        # Hold corpus if needed elsewhere
        self.chunks = None                 # <-- set in setup()

    def setup(self):
        print("加载所有页chunk...")
        self.chunks = self.loader.load_chunks()
        print(f"共加载 {len(self.chunks)} 个chunk")

        print("生成嵌入...")
        embeddings = self.embedding_model.embed_texts([c["content"] for c in self.chunks])

        print("存储向量...")
        self.vector_store.add_chunks(self.chunks, embeddings)

        # Build file-level BM25 now that chunks exist
        print("构建文件级BM25索引...")
        self.file_bm25 = FileBM25Index(pages_per_file=3, per_page_chars=1500, repeat_filename_tokens=2)
        self.file_bm25.build(self.chunks)

        print("RAG向量库构建完成！")

    def query(self, question: str, top_k: int = 3) -> Dict[str, Any]:
        q_emb = self.embedding_model.embed_text(question)
        results = self.vector_store.search(q_emb, top_k)
        return {"question": question, "chunks": results}

    def _build_context(self, items: List[Dict[str, Any]]) -> str:
        return "\n".join(
            f"[文件名]{c['metadata']['file_name']} [页码]{c['metadata']['page']}\n{c['content']}"
            for c in items
        )

    def generate_answer(self, question: str, top_k: int = 3) -> Dict[str, Any]:
        qwen_api_key = os.getenv("LOCAL_API_KEY")
        qwen_base_url = os.getenv("LOCAL_BASE_URL")
        qwen_model    = os.getenv("LOCAL_TEXT_MODEL")
        if not qwen_api_key or not qwen_base_url or not qwen_model:
            raise ValueError("请在.env中配置LOCAL_API_KEY、LOCAL_BASE_URL、LOCAL_TEXT_MODEL")

        tele = {"path":"", "rerank_ok":False, "rerank_attempts":0, "rerank_http":None,
                "cand_n":0, "ranked_n":0, "file_vote_best":None, "file_vote_n":0, "final_n":0}

        # Stage-A dense
        q_emb = self.embedding_model.embed_text(question)
        candidates = self.vector_store.search(q_emb, self.candidate_k)
        tele["cand_n"] = len(candidates)

        # Optional per-file diversity cap
        if candidates and self.limit_per_file > 0:
            by_file, diverse = {}, []
            for c in candidates:
                fn = c["metadata"]["file_name"]
                by_file.setdefault(fn, 0)
                if by_file[fn] < self.limit_per_file:
                    diverse.append(c); by_file[fn] += 1
            candidates = diverse

        # RRF fuse with file-level BM25 (only if built)
        if self.file_bm25 and candidates:
            # 记录 BEFORE (先存，再融合)
            pre_rrf_files = list(dict.fromkeys([c["metadata"]["file_name"] for c in candidates]))
            tele["first3_files_before_rrf"] = pre_rrf_files[:3]

            # 参数
            BM25_TOPN = int(os.getenv("BM25_TOPN", "25"))
            RRF_K     = int(os.getenv("RRF_K", "90"))

            # BM25 文件排名
            top_files = self.file_bm25.top_files(question, n=BM25_TOPN)
            tele["bm25_top_files"] = [fn for fn, _ in top_files[:5]]

            # 融合（统一用 RRF_K；rank 约定 0-based -> rrf 内部会 +1）
            file_to_rank = {fn: r for r, (fn, _) in enumerate(top_files)}
            BIG = 10_000

            fused = []
            for i, c in enumerate(candidates):
                fn = c["metadata"]["file_name"]
                dense_rank = i
                bm25_rank  = file_to_rank.get(fn, BIG)
                score = rrf(dense_rank, k=RRF_K) + rrf(bm25_rank, k=RRF_K)
                cc = dict(c); cc["fused_score"] = score
                fused.append(cc)

            candidates = sorted(fused, key=lambda x: x["fused_score"], reverse=True)

        # 记录 AFTER（放在 if 外也行，candidates 一定存在）
        tele["first3_files_after_rrf"] = list(dict.fromkeys([c["metadata"]["file_name"] for c in candidates]))[:3]

        # ---- Edition router: 文件级 rerank（从 Top1 改为 Top-M 保留） --------------------
        # 从 RRF 后的 candidates 里，每个文件取一个代表 chunk（首次出现即可）
        by_file_best = {}  # fn -> (score, chunk)
        for c in candidates:
            fn = c["metadata"]["file_name"]
            s  = float(c.get("rerank_score", c.get("fused_score", 0.0)))
            prev = by_file_best.get(fn)
            if (prev is None) or (s > prev[0]):
                by_file_best[fn] = (s, c)

        file_profiles = [c for _, c in by_file_best.values()]

        # 用同一个 reranker 对“文件代表”打分（它会看到 header+日期等）
        file_ranked, _ = self.reranker.rerank(question, file_profiles, return_meta=True)

        # 保留前 M 个文件（环境变量 FILE_ROUTER_TOP_M，默认 3；不足则全留）
        FILE_ROUTER_TOP_M = int(os.getenv("FILE_ROUTER_TOP_M", "3"))
        if file_ranked:
            keep_files = [c["metadata"]["file_name"] for c in file_ranked[: max(1, FILE_ROUTER_TOP_M)]]
            # 仅过滤到这几个文件，而不是只留 1 个
            candidates = [c for c in candidates if c["metadata"]["file_name"] in set(keep_files)]
            tele["file_router_topM"] = keep_files
            tele["unique_files_after_router"] = list({c["metadata"]["file_name"] for c in candidates})
        else:
            tele["file_router_topM"] = []
            tele["unique_files_after_router"] = list({c["metadata"]["file_name"] for c in candidates})
        # -----------------------------------------------------------------------




        # Rerank → soft bias → file_vote → neighbor expansion
        if self.use_rerank and self.reranker is not None and candidates:
            tele["path"] = "rerank"
            ranked, rr_meta = self.reranker.rerank(question, candidates, return_meta=True)
            tele["rerank_http"] = rr_meta.get("status"); tele["rerank_attempts"] = rr_meta.get("attempts")
            tele["ranked_n"] = len(ranked); tele["rerank_ok"] = bool(ranked)

            if not ranked:
                if self.strict_rerank:
                    raise RuntimeError(f"Rerank failed (status={tele['rerank_http']}) and STRICT_RERANK=1")
                chunks = candidates[: self.final_k]
            else:
                # 可选：微弱文件级 bias，而不是逐 chunk 大幅 bias
                ranked = soft_bias_filewise(ranked, question)
                # —— 文件投票（在保留的 Top-M 文件之间做最后拍板）——
                kept_chunks, (best_file, file_scores) = file_vote(ranked, top_n_files=1, agg="max")
                tele["file_vote_n"]     = len(file_scores)
                tele["file_vote_best"]  = best_file
                # 可选：记录前几个分数便于诊断
                tele["file_vote_scores"] = dict(list(sorted(file_scores.items(), key=lambda kv: kv[1], reverse=True))[:5])

                # 只在最佳文件内部做页级选择 + 邻域扩展
                file_scoped = [c for c in ranked if c["metadata"]["file_name"] == best_file] if best_file else ranked
                chunks = pick_best_page_and_expand(file_scoped, final_k=self.final_k, radius=1)


                # no page_vote; just take neighbors within the chosen file
                # try:
                #     chunks = expand_neighbors_on_page(file_scoped, radius=1, max_total=self.final_k)
                # except Exception:
                #     chunks = file_scoped[: self.final_k]
        else:
            tele["path"] = "baseline"
            chunks = candidates[: top_k]

        tele["final_n"] = len(chunks)

        # Early exit
        if not chunks:
            out = {"question": question, "answer": "", "filename": "", "page": "", "retrieval_chunks": []}
            if self.debug_telemetry: out["debug"] = tele
            return out

        # Evidence for LOCK_SOURCE
        top_file = chunks[0]['metadata']['file_name']
        top_page = chunks[0]['metadata']['page']

        # Build context
        context = self._build_context(chunks)

        # LLM call
        client = OpenAI(api_key=qwen_api_key, base_url=qwen_base_url)
        prompt = (
            "你是一名专业的金融分析助手，请根据以下检索到的内容回答用户问题。\n"
            "请严格按照如下JSON格式输出：\n"
            '{"answer": "你的简洁回答", "filename": "来源文件名", "page": "来源页码"}\n'
            f"检索内容：\n{context}\n\n问题：{question}\n"
            "请确保输出内容为合法JSON字符串，不要输出多余内容。"
        )
        completion = client.chat.completions.create(
            model=qwen_model,
            messages=[{"role": "system", "content": "你是一名专业的金融分析助手。"},
                      {"role": "user", "content": prompt}],
            temperature=0.2, max_tokens=1024,
        )

        import json as pyjson
        from extract_json_array import extract_json_array
        raw = completion.choices[0].message.content.strip()
        json_str = extract_json_array(raw, mode="objects")

        if json_str:
            try:
                arr = pyjson.loads(json_str)
                if isinstance(arr, list) and arr:
                    j = arr[0]
                    answer = j.get("answer", ""); filename = j.get("filename", ""); page = j.get("page", "")
                else:
                    answer, filename, page = raw, top_file, top_page
            except Exception:
                answer, filename, page = raw, top_file, top_page
        else:
            answer, filename, page = raw, top_file, top_page

        # normalize page & apply offset
        PAGE_OFFSET = int(os.getenv("PAGE_OFFSET", "0"))
        try:
            if page not in ("", None): page = int(page) + PAGE_OFFSET
        except Exception:
            try: page = int(top_page) + PAGE_OFFSET
            except Exception: page = top_page

        # LOCK_SOURCE
        model_file, model_page = filename, page
        if self.lock_source and chunks:
            filename, page = top_file, (int(top_page) + PAGE_OFFSET if str(top_page).isdigit() else top_page)

        out = {"question": question, "answer": answer, "filename": filename, "page": page, "retrieval_chunks": chunks}
        if self.debug_telemetry:
            tele.update({"top_file": top_file, "top_page": top_page, "model_file": model_file, "model_page": model_page})
            out["debug"] = tele
        return out

In [49]:
# 1) Imports & Paths
from pathlib import Path
import os, json, random
from tqdm.auto import tqdm
import concurrent.futures
import math

# Notebook is in .../notebook; project root is parent
NOTEBOOK_DIR = Path.cwd()
PROJ_ROOT = NOTEBOOK_DIR.parent

# Try common locations for train.json
CANDIDATE_TRAIN = [
    PROJ_ROOT / "datas" / "train.json",
    PROJ_ROOT / "data" / "train.json",
    NOTEBOOK_DIR / "datas" / "train.json",
    NOTEBOOK_DIR / "data" / "train.json",
]
TRAIN_PATH = next((p for p in CANDIDATE_TRAIN if p.exists()), None)
if TRAIN_PATH is None:
    raise FileNotFoundError(f"train.json not found in: {CANDIDATE_TRAIN}")

# Chunk JSON path (your earlier structure)
CHUNK_JSON_PATH = PROJ_ROOT / "notebook" / "all_pdf_windows_mineru.json"

# Outputs
EVAL_RAW_PATH = PROJ_ROOT / "eval_train_raw.json"
EVAL_SUMMARY_PATH = PROJ_ROOT / "eval_train_scored.json"

print("Notebook Dir:", NOTEBOOK_DIR)
print("Project Root :", PROJ_ROOT)
print("Train JSON   :", TRAIN_PATH)
print("Chunks JSON  :", CHUNK_JSON_PATH)

# --- RAG behavior flags (your SimpleRAG reads these) ---
env = os.environ.copy()
env.update({
    "LOCK_SOURCE":  "1",   # force filename/page to top evidence
    "STRICT_RERANK":"0",   # set "1" to fail fast if rerank API hiccups
    "DEBUG_RAG":    "1",   # include telemetry in outputs
    "LIMIT_PER_FILE": "12",
    "PAGE_OFFSET":  "0",   # MinerU 0-based -> leaderboard 1-based
    "BM25_TOPN":"25",
    "RRF_K":"90",
    "FILE_ROUTER_TOP_M":"3"
})
os.environ.update(env)


Notebook Dir: d:\Datawhale\Multimodal-RAG-Competitions\notebook
Project Root : d:\Datawhale\Multimodal-RAG-Competitions
Train JSON   : d:\Datawhale\Multimodal-RAG-Competitions\data\train.json
Chunks JSON  : d:\Datawhale\Multimodal-RAG-Competitions\notebook\all_pdf_windows_mineru.json


In [50]:
# Create your reranker beforehand
# reranker = HybridReranker(model_name="BAAI/bge-reranker-base")
reranker = SiliconFlowReranker()
rag = SimpleRAG(
    chunk_json_path=CHUNK_JSON_PATH,  # or your page/block chunk file
    use_rerank=True,
    candidate_k=120,
    final_k=5,
    reranker=reranker,
)
rag.setup()


加载所有页chunk...
共加载 10053 个chunk
生成嵌入...


Embedding: 100%|██████████| 315/315 [05:53<00:00,  1.12s/batch]


存储向量...
构建文件级BM25索引...
RAG向量库构建完成！


In [51]:
# 3) Load train and sample
with open(TRAIN_PATH, "r", encoding="utf-8") as f:
    train_data = json.load(f)

N = len(train_data)
random.seed(42)

sample_size = max(1, math.ceil(N * 0.2))
all_idx = list(range(N))
sample_idx = sorted(random.sample(all_idx, sample_size)) if sample_size < N else all_idx

print(f"Train size = {N} | Sample size = {len(sample_idx)}")
sample_idx[:10]


Train size = 118 | Sample size = 24


[3, 4, 11, 13, 14, 17, 27, 28, 29, 31]

In [52]:
# 4) Jaccard helper
def jaccard_char(a: str, b: str) -> float:
    a = (a or "").strip()
    b = (b or "").strip()
    if not a and not b:
        return 1.0
    set_a, set_b = set(a), set(b)
    union = set_a | set_b
    inter = set_a & set_b
    return len(inter) / len(union) if union else 0.0


In [53]:
# 5) Inference
def run_one(idx):
    item = train_data[idx]
    q = item.get("question", "")
    tqdm.write(f"[{sample_idx.index(idx)+1}/{len(sample_idx)}] {q[:60]}...")
    pred = rag.generate_answer(q, top_k=5)
    return idx, pred

results = []
with concurrent.futures.ThreadPoolExecutor(max_workers=10) as ex:
    for out in tqdm(ex.map(run_one, sample_idx), total=len(sample_idx), desc="Infer on train sample"):
        results.append(out)

# Save raw (idx, pred) for debugging
with open(EVAL_RAW_PATH, "w", encoding="utf-8") as f:
    json.dump(results, f, ensure_ascii=False, indent=2)
print(f"Saved raw predictions to: {EVAL_RAW_PATH}")


[1/24] 联邦制药的UBT37034在超重/肥胖适应症方面取得了哪些临床前数据？...
[2/24] 联邦制药的TUL01101片剂和软膏剂在特应性皮炎治疗中的疗效如何？...
[3/24] 极兔速递-W：2024年单位运输成本和分拣成本分别下降了多少百分比？...
[4/24] 根据华创证券对凌云股份的深度研究报告，请问该公司在2024年的主要产品收入占比是多少？...
[5/24] 关于凌云股份（600480）的热冲压技术应用和发展前景，能否详细解释热冲压成型工艺与冷冲压成型工艺的主要区别？...
[6/24] 关于凌云股份（600480）的德国WAG业务板块及客户情况，请问具体有哪些主要客户？...
[7/24] 如何分析广联达在数字化转型中的增长潜力？...
[8/24] 广联达的数字施工业务在2020年的资金压力如何？与同行业其他企业相比，其资金压力有何特点？...
[9/24] 广联达的施工总承包资质分为几个类别？每个类别分别对应哪些工程类型？...
[10/24] 如何评估广联达在数字化转型过程中面临的挑战及其应对策略？...


Infer on train sample:   0%|          | 0/24 [00:38<?, ?it/s]

[11/24] 如何评估广联达在数字化转型中的竞争优势？...
[12/24] 如何评价广联达在搅拌站材料核算系统上的核心功能及其工作流程？...


Infer on train sample:   0%|          | 0/24 [00:40<?, ?it/s]

[13/24] 广联达：根据图表217和218，您能否分析一下国家在支持工业软件和建筑业软件系统自主可控方面的政策趋势？...


...

[23/24] 广联达（002410）的数字设计业务在2021年下半年将如何推进？...


Infer on train sample:   4%|▍         | 1/24 [01:22<16:26, 42.88s/it]

[24/24] 广联达（002410）在数字建筑一体化方面有哪些具体的产品定位和解决方案？...


Infer on train sample: 100%|██████████| 24/24 [02:10<00:00,  5.43s/it]

Saved raw predictions to: d:\Datawhale\Multimodal-RAG-Competitions\eval_train_raw.json


## 6) Scoring vs Ground Truth
Score per item:
- page_match: 1 if exact page equals, else 0 (×0.25)
- filename_match: 1 if exact filename equals, else 0 (×0.25)
- answer_jaccard: char Jaccard (×0.5)


In [57]:
# 6) Score predictions
idx2pred = {idx: pred for idx, pred in results}

scored_rows = []
for idx in sample_idx:
    gt = train_data[idx]
    pred = idx2pred.get(idx, {})

    gt_q = gt.get("question", "")
    gt_a = gt.get("answer", "")
    gt_f = gt.get("filename", "")
    gt_p = gt.get("page", "")

    pr_a = pred.get("answer", "")
    pr_f = pred.get("filename", "")
    pr_p = pred.get("page", "")

    page_match = 1.0 if str(pr_p) == str(gt_p) else 0.0
    filename_match = 1.0 if str(pr_f) == str(gt_f) else 0.0
    answer_sim = jaccard_char(str(pr_a), str(gt_a))

    score = 0.25 * page_match + 0.25 * filename_match + 0.5 * answer_sim

    scored_rows.append({
        "idx": idx,
        "question": gt_q,
        "gt_answer": gt_a,
        "pr_answer": pr_a,
        "gt_filename": gt_f,
        "pr_filename": pr_f,
        "gt_page": gt_p,
        "pr_page": pr_p,
        "page_match": page_match,
        "filename_match": filename_match,
        "answer_jaccard": answer_sim,
        "score": score,
    })

# Sort by score ascending to inspect weak cases first
scored_rows_sorted = sorted(scored_rows, key=lambda x: x["score"])

with open(EVAL_SUMMARY_PATH, "w", encoding="utf-8") as f:
    json.dump(scored_rows_sorted, f, ensure_ascii=False, indent=2)

print(f"Saved scored results to: {EVAL_SUMMARY_PATH}")
print(f"max score: {max(r['score'] for r in scored_rows_sorted)}")
print(f"Mean score: {sum(r['score'] for r in scored_rows_sorted)/len(scored_rows_sorted):.4f}")
print(f"min score: {min(r['score'] for r in scored_rows_sorted) }")
print(f"Mean Jaccard: {sum(r['answer_jaccard'] for r in scored_rows_sorted)/len(scored_rows_sorted):.4f}")
print(f"Filename exact@1: {sum(r['filename_match'] for r in scored_rows_sorted)/len(scored_rows_sorted):.4f}")
print(f"Page exact@1: {sum(r['page_match'] for r in scored_rows_sorted)/len(scored_rows_sorted):.4f}")


Saved scored results to: d:\Datawhale\Multimodal-RAG-Competitions\eval_train_scored.json
max score: 0.8292682926829269
Mean score: 0.3814
min score: 0.08053691275167785
Mean Jaccard: 0.3253
Filename exact@1: 0.5000
Page exact@1: 0.3750


max score: 0.8116883116883117  
Mean score: 0.3678  
min score: 0.06818181818181818  
Mean Jaccard: 0.3118  
Filename exact@1: 0.4407  
Page exact@1: 0.4068  

Saved scored results to: d:\Datawhale\Multimodal-RAG-Competitions\eval_train_scored.json  
max score: 0.5576923076923077  
Mean score: 0.3201  
min score: 0.08403361344537816  
Mean Jaccard: 0.3068  
Filename exact@1: 0.5833  
Page exact@1: 0.0833  

Mineru + BN25
max score: 0.4962686567164179  
Mean score: 0.3231  
min score: 0.09130434782608696  
Mean Jaccard: 0.3128
Filename exact@1: 0.5833  
Page exact@1: 0.0833  

Note: With BM25 rerank :
max score: 0.5732323232323233  
Mean score: 0.2638   
min score: 0.09668508287292818  
Mean Jaccard: 0.3193  
Filename exact@1: 0.4167  
Page exact@1: 0.0000  

In [58]:
# Show a couple of worst and best cases inline (adjust k as needed)
k = 6
print("— Worst cases —")
for r in scored_rows_sorted[:k]:
    print("\nScore:", r["score"])
    print("Q:", r["question"])
    print("GT:", r["gt_answer"])
    print("PR:", r["pr_answer"])
    print("GT file/page:", r["gt_filename"], r["gt_page"])
    print("PR file/page:", r["pr_filename"], r["pr_page"])

print("\n— Best cases —")
for r in scored_rows_sorted[-k:]:
    print("\nScore:", r["score"])
    print("Q:", r["question"])
    print("GT:", r["gt_answer"])
    print("PR:", r["pr_answer"])
    print("GT file/page:", r["gt_filename"], r["gt_page"])
    print("PR file/page:", r["pr_filename"], r["pr_page"])


— Worst cases —

Score: 0.08053691275167785
Q: 广联达的数字施工业务在2020年的资金压力如何？与同行业其他企业相比，其资金压力有何特点？
GT: 根据图片中的图表和文字内容，可以得出以下结论：

1. **资金压力情况**：
   - 图表35显示，龙元建设在2015年至2020年间，应收账款占比和已完工未结算资产占比均呈现上升趋势。特别是在2015年到2016年间，应收账款占比和已完工未结算资产占比都有显著增加。

2. **与其他企业的比较**：
   - 图表36显示，宏润建设在2017年至2020年间，应收账款占比和建造合同形成的已完工未结算资产占比也
PR: 广联达数字施工业务在2020年受地产行业局部压力影响较小，因其客户需求主要来自非住宅类公共建筑和商业建筑，甲方为地方政府/城投等，财务稳定性较高。与同行业相比，其资金压力特点在于客户结构分散且非地产主导，回款风险较低。
GT file/page: 广联达-再谈广联达当前时点下如何看待其三条增长曲线-220217131页.pdf 28
PR file/page: 广联达-跟踪报告数字建筑龙头五问五答-21120815页.pdf 5

Score: 0.08430232558139535
Q: 如何评估广联达在数字化转型过程中面临的挑战及其应对策略？
GT: 广联达在数字化转型过程中面临的挑战主要包括以下几个方面：

1. **劳动力问题**：
   - 工人的体力劳动量较大，施工工作环境较苦，收入较低，导致施工人员流动性大。
   - 老龄化现象严重，30岁及以上比例的工人占比逐年提升，接近8成。
   - 受教育程度低，建筑施工以简单重复性工作为主，难以提供良好的职业上升通道。

2. **项目变化频繁**：
   - 建筑项目的设计、预算和时间表可能随时发生变化，影响工期与成本。
   - 变
PR: 广联达在数字化转型中面临技术迭代快、国际化适应及行业生态整合等挑战，通过持续研发投入、并购国际技术团队（如芬兰Progman）、发布白皮书与数字平台、强化生态合作等策略应对，推动建筑产业互联网转型。
GT file/page: 广联达-再谈广联达当前时点下如何看待其三条增长曲线-220217131页.pdf 32
PR file/page: 广联达-公司

### Notes
- Set a different **sample fraction** by changing the `0.10` in `math.ceil(N * 0.10)`.
- If `filename`/`page` in ground truth differ in minor formatting (e.g., case, spaces), add normalization before comparison.
- You can plug this same scorer later for validation on a dev split.
